In [2]:
!pip install langchain chromadb faiss-cpu  tiktoken langchain_google_genai langchain-community wikipedia

In [3]:
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

# Retriever Based on Data Source


## **Wikipedia Retriever**


In [4]:
"""This is available in community package"""
from langchain_community.retrievers import WikipediaRetriever

In [10]:
# first initialize the retiever

retriever = WikipediaRetriever(top_k_results=2)

In [11]:
# our query and result
query = "One Piece"

# get the result from retriever
result = retriever.invoke(query)

In [12]:
result

[Document(metadata={'title': 'One Piece', 'summary': 'One Piece (stylized in all caps) is a Japanese manga series written and illustrated by Eiichiro Oda. It follows the adventures of Monkey D. Luffy and his crew, the Straw Hat Pirates, as he explores the Grand Line in search of the mythical treasure known as the "One Piece" to become the next King of the Pirates.\nThe manga has been serialized in Shueisha\'s shōnen manga magazine Weekly Shōnen Jump since July 1997, with its chapters compiled in 112 tankōbon volumes as of July 2025. It was licensed for an English language release in North America and the United Kingdom by Viz Media and in Australia by Madman Entertainment. Becoming a media franchise, it has been adapted into a festival film by Production I.G, and an anime series by Toei Animation, which premiered in October 1999. Additionally, Toei has developed 14 animated feature films and one original video animation. Several companies have developed various types of merchandising a

In [ ]:
for i, doc in enumerate(result):
    print(f"======== Result {i + 1} ==========")
    print(f"Content:\n{doc.page_content}...")  # truncate for display


======== Result 1 ==========
Content:
One Piece (stylized in all caps) is a Japanese manga series written and illustrated by Eiichiro Oda. It follows the adventures of Monkey D. Luffy and his crew, the Straw Hat Pirates, as he explores the Grand Line in search of the mythical treasure known as the "One Piece" to become the next King of the Pirates.
The manga has been serialized in Shueisha's shōnen manga magazine Weekly Shōnen Jump since July 1997, with its chapters compiled in 112 tankōbon volumes as of July 2025. It was licensed for an English language release in North America and the United Kingdom by Viz Media and in Australia by Madman Entertainment. Becoming a media franchise, it has been adapted into a festival film by Production I.G, and an anime series by Toei Animation, which premiered in October 1999. Additionally, Toei has developed 14 animated feature films and one original video animation. Several companies have developed various types of merchandising and media, such as 

## **Vector Store Retreiver**


In [16]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

In [ ]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(
        page_content="Chroma is a vector database optimized for LLM-based search."
    ),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [18]:
# Step 2: Initialize embedding model
embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [ ]:
# Step 3: Create Chroma vector store in memory
vectorstore = Chroma.from_documents(
    documents=documents, embedding=embedding_model, collection_name="my_collection"
)

In [20]:
# Step 4: Convert vectorstore into a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [21]:
query = "which model is good for embeddings?"
results = retriever.invoke(query)

In [ ]:
for i, doc in enumerate(results):
    print(f"======== Result{i + 1} ========")
    print(doc.page_content)

======== Result1 ========
OpenAI provides powerful embedding models.
======== Result2 ========
Embeddings convert text into high-dimensional vectors.


Now this we have also done in previous topic named vector store like below


In [ ]:
results = vectorstore.similarity_search_with_score(query, k=2)

In [26]:
results

[(Document(metadata={}, page_content='OpenAI provides powerful embedding models.'),
  0.20905672013759613),
 (Document(metadata={}, page_content='Embeddings convert text into high-dimensional vectors.'),
  0.2560487389564514)]

In [ ]:
for i, doc in enumerate(results):
    print(f"======== Result{i + 1} ========")
    print(doc)

======== Result1 ========
(Document(metadata={}, page_content='OpenAI provides powerful embedding models.'), 0.20905672013759613)
======== Result2 ========
(Document(metadata={}, page_content='Embeddings convert text into high-dimensional vectors.'), 0.2560487389564514)


SO now what is the difference between vector store similarity search and retrievers.

1. Vector Store has a fixed search strategy
2. So if we need to apply any other searching mechanism we need to have create retrievers
3. Since Retriever is also a Runnable it is usefull for Connecting with Different Components


# Retriever Based on Searching Mechanism


## **MMR**


In [39]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(
        page_content="MMR helps you get diverse results when doing similarity search."
    ),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [ ]:
# Initialize OpenAI embeddings
embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")


In [42]:
# Create FAISS vector store
vectorstore = FAISS.from_documents(documents=docs, embedding=embedding_model)

In [ ]:
# Create retrievers of MMR
retriever = vectorstore.as_retriever(
    search_type="mmr",  # this means search type mmr
    # k=top result, lambda_mult = relavance-diversity balance
    search_kwargs={"k": 3, "lambda_mult": 0.5},
)

In [ ]:
query = "What is Langchain"

result = retriever.invoke(query)

In [ ]:
for i, doc in enumerate(result):
    print(doc.page_content)

LangChain is used to build LLM based applications.
Embeddings are vector representations of text.
LangChain supports Chroma, FAISS, Pinecone, and more.


## **Multi Query Retriever**


In [46]:
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# Relevant health & wellness documents
all_docs = [
    Document(
        page_content="Regular walking boosts heart health and can reduce symptoms of depression.",
        metadata={"source": "H1"},
    ),
    Document(
        page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.",
        metadata={"source": "H2"},
    ),
    Document(
        page_content="Deep sleep is crucial for cellular repair and emotional regulation.",
        metadata={"source": "H3"},
    ),
    Document(
        page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.",
        metadata={"source": "H4"},
    ),
    Document(
        page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.",
        metadata={"source": "H5"},
    ),
    Document(
        page_content="The solar energy system in modern homes helps balance electricity demand.",
        metadata={"source": "I1"},
    ),
    Document(
        page_content="Python balances readability with power, making it a popular system design language.",
        metadata={"source": "I2"},
    ),
    Document(
        page_content="Photosynthesis enables plants to produce energy by converting sunlight.",
        metadata={"source": "I3"},
    ),
    Document(
        page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.",
        metadata={"source": "I4"},
    ),
    Document(
        page_content="Black holes bend spacetime and store immense gravitational energy.",
        metadata={"source": "I5"},
    ),
]

In [ ]:
# Initialize OpenAI embeddings
embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# Create FAISS vector store
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embedding_model)

In [ ]:
# Create retrievers Basic
similarity_retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 5}
)

In [ ]:
# Retriever Multi Query
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash"),
)

In [ ]:
# Query
query = "How to improve energy and maintain balance"

In [ ]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results = multiquery_retriever.invoke(query)

In [ ]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i + 1} ---")
    print(doc.page_content)

print("*" * 150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i + 1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 4 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 5 ---
Deep sleep is crucial for cellular repair and emotional regulation.
******************************************************************************************************************************************************

--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 3 ---
Deep sleep is crucial for cellular repair and emotional regulation.

--- Result 4 ---
Mindfulness and controlled breathing lower cortisol and improve mental clar

## **Contextual Compression Retriever**


In [54]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [ ]:
docs = [
    Document(
        page_content=(
            """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
        ),
        metadata={"source": "Doc1"},
    ),
    Document(
        page_content=(
            """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
        ),
        metadata={"source": "Doc2"},
    ),
    Document(
        page_content=(
            """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
        ),
        metadata={"source": "Doc3"},
    ),
    Document(
        page_content=(
            """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
        ),
        metadata={"source": "Doc4"},
    ),
]

In [ ]:
# Create a FAISS vector store from the documents
embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vectorstore = FAISS.from_documents(docs, embedding_model)

In [58]:
# Retrieve Relevant docs
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [59]:
# Set up the compressor using an LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
# Create a compression using llm i.e trim the docs
compressor = LLMChainExtractor.from_llm(llm)

In [ ]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    # How to retrieve
    base_retriever=base_retriever,
    # how to compress
    base_compressor=compressor,
)

In [61]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [ ]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i + 1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
Photosynthesis does not occur in animal cells.

--- Result 3 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.


https://python.langchain.com/docs/integrations/retrievers/
